In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk import sent_tokenize
import torch
from sentence_transformers import SentenceTransformer, util



#Similarity search for car make model

The current problem of the existing SgCarMart search engine is the less accurate search results from users’ inputs that might affect customer experiences. For example, when a user types ‘cc’ in the search box, the returned result is usually Honda Accord at the top, instead of Volkswagen cc.

Especially while you type the model, mostly getting the results is not accurate.

I hope AI will help us the problem

mount the drive

In [ ]:
df=pd.read_csv('/content/gdrive/My Drive/ISY5002/Project/Image_Search/data/car_make model_2.csv')
df['CarModel']= df['CarModel'].apply(lambda x: x.lower())

In [ ]:
corpus = df['CarModel'].astype(str).values.tolist()

In [ ]:
#put the search terms, and corpus
base_document = corpus
query = 'cc'
#query = 'bmw'

#threhold
highest_score = 1
lowest_score = 0.55

#top result
top_k = 5


In [ ]:
import operator
from itertools import islice


def get_best_similar(best_suit):
  sorted_d = dict( sorted(best_suit.items(), key=operator.itemgetter(1),reverse=True))
  return (dict(islice(sorted_d.items(), top_k)))


BERT

We came out with solution to solve this issue is using BERT pre-train model.

What is BERT?

**BERT** (introduced in [this paper](https://arxiv.org/abs/1810.04805)) stands for Bidirectional Encoder Representations from Transformers. It is a trained Transformer Encoder stack. BERT is a model that broke several records for how well models can handle language-based tasks after the release paper. It is saving the time, energy, knowledge, and resources that would have gone to training a language-processing model from scratch. Its the good point.

The paper presents two model sizes for BERT:
BERT BASE (L=12, H=768, A=12, Total Parameters=110M) and BERT LARGE (L=24, H=1024, A=16, Total Parameters=340M).In this project, i will use bert-base. 


more clear explaination in [it](https://jalammar.github.io/illustrated-bert/)

In [ ]:
def cosine_similarities_test(query_embedding, corpus_embeddings, top_k=5):
	cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
	top_results = torch.topk(cos_scores, k=top_k)
	for score, idx in zip(top_results[0], top_results[1]):
		print(corpus[idx], "(Score: {:.4f})".format(score))


In [ ]:
def get_bert_similarity(query, base_document):
	# This will download and load the pretrained model offered by UKPLab.
	model = SentenceTransformer('all-MiniLM-L6-v2')

	recommendations=[]

	# Although it is not explicitly stated in the official document of sentence transformer, the original BERT is meant for a shorter sentence. We will feed the model by sentences instead of the whole documents.
	base_embeddings_sentences = model.encode(base_document)
	for document in [query]:
		embeddings_sentences = model.encode(document)
		cos_scores = util.pytorch_cos_sim(embeddings_sentences, base_embeddings_sentences)[0]
		top_results = torch.topk(cos_scores, k=top_k)
		print("\n\n======================\n\n")
		print("Query:", document)
		print("\nTop 5 most similar sentences in corpus:")
		for score, idx in zip(top_results[0], top_results[1]):
			if score > 0.5:
				recommendations.append(corpus[idx])

		print(corpus[idx], "(Score: {:.4f})".format(score))



	return recommendations
